## Data Cleaning Project

In [218]:
import pandas as pd
import numpy as np
from scipy import stats
pd.set_option('display.float_format', lambda x: '%.2f' % x) ##suppres sci notation

In [219]:
##read in the file
autos = pd.read_csv("autos.csv", encoding = 'Latin-1')

In [220]:
autos.head(5)

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [221]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

We can see that most of the variables are coded as objects.  However, some variable types should be converted; for example, the date variables should be converted to timestamp, and price and odometer should be integer values.  In addition, to follow Python convention, the column names should be converted from camelcase to snakecase.

In [222]:
#list of columns to see what should be changed
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [223]:
##create a copy of the column list to change,set all to lowercase and add underscore between words
col_copy = autos.columns
col_copy = col_copy.str.lower()

In [224]:
col_copy = col_copy.str.replace("datecrawled", "date_crawled")
col_copy = col_copy.str.replace("datecreated", "ad_created")
col_copy = col_copy.str.replace("offertype", "offer_type")
col_copy = col_copy.str.replace("vehicletype", "vehicle_type")
col_copy = col_copy.str.replace("fueltype", "fuel_type")
col_copy = col_copy.str.replace("nrofpicture", "nr_of_picture")
col_copy = col_copy.str.replace("postalcode", "postal_code")
col_copy = col_copy.str.replace("gearbox", "gear_box")
col_copy = col_copy.str.replace("lastseen", "last_seen")
col_copy = col_copy.str.replace("abtest", "ab_test")
col_copy = col_copy.str.replace("yearofregistration", "registration_year")
col_copy = col_copy.str.replace("monthofregistration", "registration_month")

In [225]:
autos.columns = col_copy

Now I'll explore some of the columns that I think should have different datatypes.

In [226]:
autos['price'].head(5)

0    $5,000
1    $8,500
2    $8,990
3    $4,350
4    $1,350
Name: price, dtype: object

In [227]:
##remove '$' and ',' from the price column
autos['price'] = autos['price'].str.replace("$","")
autos['price'] = autos['price'].str.replace(",","")

In [228]:
autos['price'].head(5)

0    5000
1    8500
2    8990
3    4350
4    1350
Name: price, dtype: object

I'll double check that there are no decimal points in this column, so I can be sure in converting it to int and not float.  

In [229]:
autos[autos["price"].str.contains("\.")==True].shape

(0, 20)

In [230]:
autos['price'] = autos['price'].astype(int)

Looking at the odometer column, I can see that the km symbol and the comma need to be removed.

In [231]:
autos['odometer'].head(5)

0    150,000km
1    150,000km
2     70,000km
3     70,000km
4    150,000km
Name: odometer, dtype: object

In [232]:
autos['odometer'] = autos['odometer'].str.replace("km", "")
autos['odometer'] = autos['odometer'].str.replace(",", "")

In [233]:
autos['odometer'].head(5)

0    150000
1    150000
2     70000
3     70000
4    150000
Name: odometer, dtype: object

In [234]:
autos['odometer'] = autos['odometer'].astype(int)

What other columns should be transformed?

In [235]:
autos.dtypes

date_crawled          object
name                  object
seller                object
offer_type            object
price                  int64
ab_test               object
vehicle_type          object
registration_year      int64
gear_box              object
powerps                int64
model                 object
odometer               int64
registration_month     int64
fuel_type             object
brand                 object
notrepaireddamage     object
ad_created            object
nr_of_pictures         int64
postal_code            int64
last_seen             object
dtype: object

The date columns, date_crawled and ad_created, should be converted to datetime

In [236]:
autos['date_crawled'] = pd.to_datetime(autos['date_crawled'])
autos['ad_created'] = pd.to_datetime(autos['ad_created'])
autos['last_seen'] = pd.to_datetime(autos['last_seen'])

In [237]:
autos.dtypes

date_crawled          datetime64[ns]
name                          object
seller                        object
offer_type                    object
price                          int64
ab_test                       object
vehicle_type                  object
registration_year              int64
gear_box                      object
powerps                        int64
model                         object
odometer                       int64
registration_month             int64
fuel_type                     object
brand                         object
notrepaireddamage             object
ad_created            datetime64[ns]
nr_of_pictures                 int64
postal_code                    int64
last_seen             datetime64[ns]
dtype: object

At this point, the columns are the correct type, so I'll now get an overview of the dataframe and look for any possible errors or irregularities

In [238]:
autos.describe(include="all")

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gear_box,powerps,model,odometer,registration_month,fuel_type,brand,notrepaireddamage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000.00,50000,44905,50000.00,47320,50000.00,47242,50000.00,50000.00,45518,50000,40171,50000,50000.00,50000.00,50000
unique,48213,38754,2,2,nan,2,8,nan,2,nan,245,nan,nan,7,40,2,76,nan,nan,39481
top,2016-03-12 16:06:22,Ford_Fiesta,privat,Angebot,nan,test,limousine,nan,manuell,nan,golf,nan,nan,benzin,volkswagen,nein,2016-04-03 00:00:00,nan,nan,2016-04-07 06:17:27
freq,3,78,49999,49999,nan,25756,12859,nan,36993,nan,4024,nan,nan,30107,10687,35232,1946,nan,nan,8
first,2016-03-05 14:06:30,NaN,NaN,NaN,nan,NaN,NaN,nan,NaN,nan,NaN,nan,nan,NaN,NaN,NaN,2015-06-11 00:00:00,nan,nan,2016-03-05 14:45:46
last,2016-04-07 14:36:56,NaN,NaN,NaN,nan,NaN,NaN,nan,NaN,nan,NaN,nan,nan,NaN,NaN,NaN,2016-04-07 00:00:00,nan,nan,2016-04-07 14:58:50
mean,NaN,NaN,NaN,NaN,9840.04,NaN,NaN,2005.07,NaN,116.36,NaN,125732.70,5.72,NaN,NaN,NaN,NaN,0.00,50813.63,NaN
std,NaN,NaN,NaN,NaN,481104.38,NaN,NaN,105.71,NaN,209.22,NaN,40042.21,3.71,NaN,NaN,NaN,NaN,0.00,25779.75,NaN
min,NaN,NaN,NaN,NaN,0.00,NaN,NaN,1000.00,NaN,0.00,NaN,5000.00,0.00,NaN,NaN,NaN,NaN,0.00,1067.00,NaN
25%,NaN,NaN,NaN,NaN,1100.00,NaN,NaN,1999.00,NaN,70.00,NaN,125000.00,3.00,NaN,NaN,NaN,NaN,0.00,30451.00,NaN


Some of the columns, such as 'seller', 'offer_type', 'gear_box', 'nr_of_pictures' and 'unrepaired_damage' have little variablity.  In fact, 'nr_of_pictures' has only 0 values, so this column can definitely be dropped.  Depending on the analysis, the other columns mentioned may be discarded.  

In [239]:
autos.drop(['nr_of_pictures'], axis=1, inplace=True)

There are also clear irregularities in the data that can be seen from the above summary.  

Look at summary over dataframe to get overview and spot any potential problems.  The highest-priced vehicle is close to 100 million dollars, which seems absurd.  What vehicle is this?

In [240]:
autos[autos['price']==99999999]

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gear_box,powerps,model,odometer,registration_month,fuel_type,brand,notrepaireddamage,ad_created,postal_code,last_seen
39705,2016-03-22 14:58:27,Tausch_gegen_gleichwertiges,privat,Angebot,99999999,control,limousine,1999,automatik,224,s_klasse,150000,9,benzin,mercedes_benz,NaN,2016-03-22,73525,2016-04-06 05:15:30


The car in question is a Mercedes S-Class limosuine, registered in 1999.  Even new models, while expensive, don't seem to cost more than $150,000.  So clearly this is an error.  

The lack of variety in the odometer column is suspicious, as this is a continuous column that should not have such a small count of unique values.  However, we can see that this is currently the case. 

In [241]:
autos['odometer'].value_counts()

150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
5000        967
40000       819
30000       789
20000       784
10000       264
Name: odometer, dtype: int64

There are only 13 different odometer values, and they are all multiples of 1,000.  This is very suspicious and suggests a data error--perhaps sloppy data entry.  

The 'powerps' column, also has a huge outlier which is clearly an error.  Currently, the highest ps (horsepower) on the market is about 916.  The highest value 17,700 is well over ten times that, and is clearly an error.  Let's see what car it corresponds to.  

In [242]:
autos[autos['powerps']==17700]

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gear_box,powerps,model,odometer,registration_month,fuel_type,brand,notrepaireddamage,ad_created,postal_code,last_seen
36421,2016-03-20 16:51:48,Verkaufe_meinen_bmw_525d,privat,Angebot,6000,test,kombi,2004,automatik,17700,5er,150000,7,diesel,bmw,nein,2016-03-20,26316,2016-04-06 22:45:56


A quick internet search shows that the BMW Kombi has a PS around 230, so the value in the table is clearly an error.  

Because I cannot trace the source of the errors in this table, I'll try to remove some outliers before my analysis.  

While there is no exact definition of what consitutes an outlier, a rule often used is values that are 1.5*IQR below or above the third and first quartiles.  That is the rule I'll use for the price, odometer and powerps columns

In [243]:
first_quart = autos['price'].quantile([.25]).at[.25]
third_quart  = autos['price'].quantile([.75]).at[.75]
IQR = third_quart - first_quart
outlier_lower_bound = (first_quart - (1.5*IQR))
outlier_upper_bound = (third_quart + (1.5*IQR))

In [244]:
print(outlier_lower_bound,outlier_upper_bound)

-8050.0 16350.0


Of course, there is no data with a negative price, so let's just look at what has a price greater than 16350.  

In [245]:
autos[autos['price']>16350].shape

(3784, 19)

Using the IQR method, this will remove about 7.5% of the data.  We can also try the Z-score method, where the data is normalized and then the data points with Z-scores above 3 and below -3 are removed   

In [246]:
autos['z_price'] = np.abs(stats.zscore(autos['price']))

In [247]:
autos[autos['z_price']>3]

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gear_box,powerps,model,odometer,registration_month,fuel_type,brand,notrepaireddamage,ad_created,postal_code,last_seen,z_price
2897,2016-03-12 21:50:57,Escort_MK_1_Hundeknochen_zum_umbauen_auf_RS_2000,privat,Angebot,11111111,test,limousine,1973,manuell,48,escort,50000,3,benzin,ford,nein,2016-03-12,94469,2016-03-12 22:45:27,23.07
11137,2016-03-29 23:52:57,suche_maserati_3200_gt_Zustand_unwichtig_laufe...,privat,Angebot,10000000,control,coupe,1960,manuell,368,NaN,100000,1,benzin,sonstige_autos,nein,2016-03-29,73033,2016-04-06 21:18:11,20.77
24384,2016-03-21 13:57:51,Schlachte_Golf_3_gt_tdi,privat,Angebot,11111111,test,NaN,1995,NaN,0,NaN,150000,0,NaN,volkswagen,NaN,2016-03-21,18519,2016-03-21 14:40:18,23.07
27371,2016-03-09 15:45:47,Fiat_Punto,privat,Angebot,12345678,control,NaN,2017,NaN,95,punto,150000,0,NaN,fiat,NaN,2016-03-09,96110,2016-03-09 15:45:47,25.64
39377,2016-03-08 23:53:51,Tausche_volvo_v40_gegen_van,privat,Angebot,12345678,control,NaN,2018,manuell,95,v40,150000,6,NaN,volvo,nein,2016-03-08,14542,2016-04-06 23:17:31,25.64
39705,2016-03-22 14:58:27,Tausch_gegen_gleichwertiges,privat,Angebot,99999999,control,limousine,1999,automatik,224,s_klasse,150000,9,benzin,mercedes_benz,NaN,2016-03-22,73525,2016-04-06 05:15:30,207.84
42221,2016-03-08 20:39:05,Leasinguebernahme,privat,Angebot,27322222,control,limousine,2014,manuell,163,c4,40000,2,diesel,citroen,NaN,2016-03-08,76532,2016-03-08 20:39:05,56.77
47598,2016-03-31 18:56:54,Opel_Vectra_B_1_6i_16V_Facelift_Tuning_Showcar...,privat,Angebot,12345678,control,limousine,2001,manuell,101,vectra,150000,3,benzin,opel,nein,2016-03-31,4356,2016-03-31 18:56:54,25.64
47634,2016-04-04 21:25:21,Ferrari_FXX,privat,Angebot,3890000,test,coupe,2006,NaN,799,NaN,5000,7,NaN,sonstige_autos,nein,2016-04-04,60313,2016-04-05 12:07:37,8.07


This method leads to a lot less data loss, let's remove these observations first and look again at the description of the price Series.  

In [248]:
drop_idx = autos[autos['z_price']>3].index
autos.drop(drop_idx, inplace=True)

In [249]:
autos['price'].describe()

count     49991.00
mean       5831.66
std       14273.37
min           0.00
25%        1100.00
50%        2950.00
75%        7200.00
max     1300000.00
Name: price, dtype: float64

In [250]:
autos[autos['price'] ==1300000]

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gear_box,powerps,model,odometer,registration_month,fuel_type,brand,notrepaireddamage,ad_created,postal_code,last_seen,z_price
7814,2016-04-04 11:53:31,Ferrari_F40,privat,Angebot,1300000,control,coupe,1992,NaN,0,NaN,50000,12,NaN,sonstige_autos,nein,2016-04-04,60598,2016-04-05 11:34:11,2.68


Now, the highest priced car is a Ferrari F40, which is indeed a very expensive sports car, and some have sold for more than 1 million dollars.  So, now I feel more confident that the values in the price column are more reasonable, and I turn to powerps  

In [251]:
autos['z_ps'] = np.abs(stats.zscore(autos['powerps']))

In [252]:
print(autos[autos['z_ps']>3].shape)

(64, 21)


In [253]:
drop_idx = autos[autos['z_ps']>3].index
autos.drop(drop_idx, inplace=True)

In [254]:
autos['powerps'].describe()

count   49927.00
mean      112.06
std        70.66
min         0.00
25%        69.00
50%       105.00
75%       150.00
max       740.00
Name: powerps, dtype: float64

Now the powerps column has much more reasonable values as well.  

What about the odometer column? It's within reason to have an odometer with 150,000 KMs recorded.  However, it's worth seeing if I come up with outliers using the z-score method.  

In [255]:
autos['z_odom'] = np.abs(stats.zscore(autos['odometer']))

In [256]:
print(autos[autos['z_odom']>3].shape)

(958, 22)


In [257]:
autos[autos['z_odom']>3]['odometer'].value_counts()

5000    958
Name: odometer, dtype: int64

So, I see that 5,000 is an outlier on the lower end, however, I see no reason to remove such rows.  Many cars for sale have little mileage on them.  

In [258]:
autos['last_seen'].dt.date.value_counts(normalize=True, dropna=False)

2016-04-06   0.22
2016-04-07   0.13
2016-04-05   0.12
2016-03-17   0.03
2016-04-03   0.03
2016-04-02   0.02
2016-03-30   0.02
2016-04-04   0.02
2016-03-31   0.02
2016-03-12   0.02
2016-04-01   0.02
2016-03-29   0.02
2016-03-22   0.02
2016-03-28   0.02
2016-03-21   0.02
2016-03-20   0.02
2016-03-24   0.02
2016-03-25   0.02
2016-03-23   0.02
2016-03-26   0.02
2016-03-16   0.02
2016-03-27   0.02
2016-03-15   0.02
2016-03-19   0.02
2016-03-14   0.01
2016-03-11   0.01
2016-03-10   0.01
2016-03-09   0.01
2016-03-13   0.01
2016-03-08   0.01
2016-03-18   0.01
2016-03-07   0.01
2016-03-06   0.00
2016-03-05   0.00
Name: last_seen, dtype: float64

In [259]:
autos['ad_created'].dt.date.value_counts(normalize=True, dropna=False)

2016-04-03   0.04
2016-03-20   0.04
2016-03-21   0.04
2016-04-04   0.04
2016-03-12   0.04
2016-03-14   0.04
2016-04-02   0.04
2016-03-28   0.03
2016-03-07   0.03
2016-03-29   0.03
2016-03-19   0.03
2016-04-01   0.03
2016-03-15   0.03
2016-03-30   0.03
2016-03-08   0.03
2016-03-09   0.03
2016-03-11   0.03
2016-03-22   0.03
2016-03-26   0.03
2016-03-23   0.03
2016-03-31   0.03
2016-03-25   0.03
2016-03-10   0.03
2016-03-17   0.03
2016-03-27   0.03
2016-03-16   0.03
2016-03-24   0.03
2016-03-05   0.02
2016-03-13   0.02
2016-03-06   0.02
             ... 
2016-02-19   0.00
2016-02-20   0.00
2016-02-14   0.00
2016-01-10   0.00
2016-02-18   0.00
2016-02-24   0.00
2016-02-26   0.00
2016-02-02   0.00
2016-02-05   0.00
2016-02-09   0.00
2016-01-29   0.00
2015-09-09   0.00
2016-02-01   0.00
2016-02-11   0.00
2016-01-22   0.00
2016-02-08   0.00
2015-06-11   0.00
2016-01-13   0.00
2016-01-03   0.00
2015-08-10   0.00
2016-02-16   0.00
2015-11-10   0.00
2016-02-22   0.00
2016-01-07   0.00
2016-02-17

In [260]:
autos['date_crawled'].dt.date.value_counts(normalize=True, dropna=False)

2016-04-03   0.04
2016-03-20   0.04
2016-03-21   0.04
2016-03-12   0.04
2016-03-14   0.04
2016-04-04   0.04
2016-03-07   0.04
2016-04-02   0.04
2016-03-19   0.03
2016-03-28   0.03
2016-03-29   0.03
2016-03-15   0.03
2016-04-01   0.03
2016-03-30   0.03
2016-03-08   0.03
2016-03-09   0.03
2016-03-22   0.03
2016-03-11   0.03
2016-03-26   0.03
2016-03-23   0.03
2016-03-10   0.03
2016-03-31   0.03
2016-03-25   0.03
2016-03-17   0.03
2016-03-27   0.03
2016-03-16   0.03
2016-03-24   0.03
2016-03-05   0.03
2016-03-13   0.02
2016-03-06   0.01
2016-04-05   0.01
2016-03-18   0.01
2016-04-06   0.00
2016-04-07   0.00
Name: date_crawled, dtype: float64

In [266]:
autos['registration_year'].value_counts(normalize=True)[0:20]

2000   0.07
2005   0.06
1999   0.06
2004   0.05
2003   0.05
2006   0.05
2001   0.05
2002   0.05
1998   0.05
2007   0.05
2008   0.04
2009   0.04
1997   0.04
2011   0.03
2010   0.03
2017   0.03
1996   0.03
2012   0.03
2016   0.03
1995   0.03
Name: registration_year, dtype: float64

In [265]:
autos['registration_month'].value_counts()

3     5065
0     5065
6     4361
5     4102
4     4097
7     3944
10    3649
12    3435
9     3385
11    3357
1     3278
8     3188
2     3001
Name: registration_month, dtype: int64

In [267]:
autos['registration_month'].unique()

array([ 3,  6,  7,  4,  8, 12, 10,  0,  9, 11,  5,  2,  1])

In [268]:
autos['registration_year'].unique()

array([2004, 1997, 2009, 2007, 2003, 2006, 1995, 1998, 2000, 2017, 2010,
       1999, 1982, 1990, 2015, 2014, 1996, 1992, 2005, 2002, 2012, 2011,
       2008, 1985, 2016, 1994, 1986, 2001, 2018, 2013, 1972, 1993, 1988,
       1989, 1967, 1973, 1956, 1976, 4500, 1987, 1991, 1983, 1960, 1969,
       1950, 1978, 1980, 1984, 1963, 1977, 1961, 1968, 1934, 1965, 1971,
       1966, 1979, 1981, 1970, 1974, 1910, 1975, 5000, 4100, 2019, 1959,
       9996, 9999, 6200, 1964, 1958, 1948, 1931, 1943, 9000, 1941, 1962,
       1927, 1937, 1929, 1000, 1957, 1952, 1111, 1955, 1939, 8888, 1954,
       1938, 2800, 5911, 1500, 1953, 1951, 4800, 1001])

Regarding the date variables, there are a few things to note. Most cars in the dataset have been registered in the late 90's or after 2000.  March and June are the months when most registrations occur.  However, we also see a value of 0 in the months, which I assume is entered when the month data isn't available.  The last_seen date is much higher for later dates, which makes the most sense.

There are some clear data errors in registration year.  For example, the values 1001 and 4800 are present, both impossible for vehicle registration years.  The data was pulled in 2016, so it seems reasonable to weed out observations with registration years outside of the 1900-2016 range (the first car was invented in 1885).  

In [272]:
drop_idx = autos[~autos['registration_year'].between(1900,2016,inclusive=True)].index
autos.drop(drop_idx, inplace=True)

In [274]:
autos['registration_year'].describe()

count   47965.00
mean     2002.81
std         7.31
min      1910.00
25%      1999.00
50%      2003.00
75%      2008.00
max      2016.00
Name: registration_year, dtype: float64

In [273]:
autos['registration_year'].value_counts(normalize=True)

2000   0.07
2005   0.06
1999   0.06
2004   0.06
2003   0.06
2006   0.06
2001   0.06
2002   0.05
1998   0.05
2007   0.05
2008   0.05
2009   0.04
1997   0.04
2011   0.03
2010   0.03
1996   0.03
2012   0.03
2016   0.03
1995   0.03
2013   0.02
2014   0.01
1994   0.01
1993   0.01
2015   0.01
1990   0.01
1992   0.01
1991   0.01
1989   0.00
1988   0.00
1985   0.00
       ... 
1966   0.00
1977   0.00
1975   0.00
1969   0.00
1965   0.00
1964   0.00
1910   0.00
1963   0.00
1959   0.00
1961   0.00
1956   0.00
1958   0.00
1937   0.00
1962   0.00
1950   0.00
1954   0.00
1941   0.00
1934   0.00
1957   0.00
1951   0.00
1955   0.00
1931   0.00
1953   0.00
1943   0.00
1938   0.00
1939   0.00
1927   0.00
1929   0.00
1948   0.00
1952   0.00
Name: registration_year, Length: 78, dtype: float64

We now see that the max value of registration_year is 2016 and the min 1910.  In addition, it's still the case that most of the cars in the dataset were registered from 1990 onward.  

## Aggregation

In [300]:
brand_mask = autos['brand'].value_counts(normalize=True)>=.05
popular_brands = brand_mask[brand_mask==True].index.tolist()

In [303]:
autos.loc[autos['brand']=='ford', 'price'].mean()

3953.517200119653

In [318]:
##for the popular brands, get mean price
brand_mean_price_dict ={}
for brand in popular_brands:
    mean_price = autos.loc[autos['brand']==brand, 'price'].mean()
    brand_mean_price_dict[brand] = mean_price
print(brand_mean_price_dict)

{'audi': 9086.707840772015, 'ford': 3953.517200119653, 'bmw': 8336.321908729407, 'opel': 2874.969720347155, 'volkswagen': 5427.8745209786775, 'mercedes_benz': 8492.077848239667}


So, out of the most popular brands, I can see that Audi's have the highest mean price.  The lowest priced car one average is the opel.  Ford's are the second-lowest. 

In [323]:
mean_price_series = pd.Series(brand_mean_price_dict)

In [324]:
mean_df = pd.DataFrame(mean_price_series, columns=['mean_price'])

In [321]:
##get mean mileage
##for the popular brands, get mean price
brand_mean_km_dict ={}
for brand in popular_brands:
    mean_km = autos.loc[autos['brand']==brand, 'odometer'].mean()
    brand_mean_km_dict[brand] = mean_km
print(brand_mean_km_dict)

{'audi': 129311.21833534379, 'ford': 124074.18486389471, 'bmw': 132424.73016474152, 'opel': 129273.86692381871, 'volkswagen': 128739.31413972683, 'mercedes_benz': 130848.45834244478}


In [325]:
mean_km_series = pd.Series(brand_mean_km_dict)

In [326]:
mean_df['mean_km']= mean_km_series

In [328]:
mean_df.sort_values(['mean_price'])

,mean_price,mean_km
opel,2874.97,129273.87
ford,3953.52,124074.18
volkswagen,5427.87,128739.31
bmw,8336.32,132424.73
mercedes_benz,8492.08,130848.46
audi,9086.71,129311.22


If we look at the correlation between mean_price and mean_km, nothing readily jumps out.  The lowest priced and highest priced cars have close mean_km values.  The highest mean_km value vehicle is the BMW, which is the lowest-priced top-tier vehicle.  Because of what we found previously with the odometer values--that they're suspiciously all round multiples of 1,000 and there are only 13 unique values--it's not surprising to see a lack of correlation.  